## IMPORT 

In [136]:
import re
import nltk
import pandas as pd
import numpy as np
from unicode_tr import unicode_tr
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## READ DATA


In [54]:
path = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj").iterdir()
# spor_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/spor").iterdir()
# ekonomi_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/ekonomi").iterdir()
# teknoloji_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/teknoloji").iterdir()
# saglik_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/saglik").iterdir()
# siyaset_data = Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/siyaset").iterdir()


In [55]:
print(list(Path("/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj").iterdir()))

[PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/spor'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/teknoloji'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/ekonomi'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/saglik'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/kultursanat'), PosixPath('/home/emir/MyProjects/DataI_Project/data/TTC-3600_Orj/siyaset')]


In [59]:
data_list = []
data_df = pd.DataFrame()
for i in path:
    for j in i.iterdir():
        data = j.read_text()
        data_list.append((str(i).split("/")[-1],data))

In [60]:
data_df = pd.DataFrame(data_list, columns=["label","text"])

In [61]:
data_df

,label,text
0,spor,Mehmet Topal'ın eşinden örnek davranış\nFenerb...
1,spor,﻿\n\tSezonun tamamlanmasının ardından Aytaç Öd...
2,spor,﻿Geride bıraktığımız \n sezon \...
3,spor,"﻿TRABZON - İbrahim Hacıosmanoğlu, Trabzonspor ..."
4,spor,Kunter: 'Beşiktaş'ın hedefi şampiyonluk'\nBeşi...
...,...,...
3595,siyaset,11 milyon tablet geliyor\n11 milyon tabletin i...
3596,siyaset,﻿\n\tAynı anda kayıt yaptıran Bakan Kılıç ile ...
3597,siyaset,﻿\n\n \n \n \n \n \n\tHDP H...
3598,siyaset,AK Parti pankartına CHP tepkisi\nAdalet ve Kal...


In [62]:
data_df["label"].unique()

array(['spor', 'teknoloji', 'ekonomi', 'saglik', 'kultursanat', 'siyaset'],
      dtype=object)

In [63]:
mapping = {'spor':0, 
           'teknoloji':1, 
           'ekonomi':2, 
           'saglik':3, 
           'kultursanat':4, 
           'siyaset':5
          }

In [64]:
data_df['label'] = data_df['label'].map(mapping)
data_df

,label,text
0,0,Mehmet Topal'ın eşinden örnek davranış\nFenerb...
1,0,﻿\n\tSezonun tamamlanmasının ardından Aytaç Öd...
2,0,﻿Geride bıraktığımız \n sezon \...
3,0,"﻿TRABZON - İbrahim Hacıosmanoğlu, Trabzonspor ..."
4,0,Kunter: 'Beşiktaş'ın hedefi şampiyonluk'\nBeşi...
...,...,...
3595,5,11 milyon tablet geliyor\n11 milyon tabletin i...
3596,5,﻿\n\tAynı anda kayıt yaptıran Bakan Kılıç ile ...
3597,5,﻿\n\n \n \n \n \n \n\tHDP H...
3598,5,AK Parti pankartına CHP tepkisi\nAdalet ve Kal...


## PREPROCESSING

In [65]:
text_list = []
for text in data_df.text:
    text = unicode_tr(text).lower()
    text = re.sub(r"[^a-zğüı'şöç]", " ", text)
    text_list.append(text)

In [85]:
max_features = 15000
count_vectorizer = CountVectorizer(max_features = max_features)
count_vectorizer.fit(text_list)
sparce_matrix = count_vectorizer.transform(text_list)

In [86]:
y= data_df.iloc[:,0].values
x= sparce_matrix.toarray()

## TRAIN TEST SPLIT

In [87]:
X_train,X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20, random_state = 42)

## CREATE MODEL and TRAINING

In [88]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## PREDICT

In [89]:
# y_pred = nb.predict(X_test)
print("accurancy: ", nb.score(X_test,Y_test))

accurancy:  0.9333333333333333


## test new sentences


In [126]:
text1 = ["Antalya'nın Muratpaşa ilçesinde yaşayan emekli Ahmet Turan Gürel de sunulan kredi avantajından haberdar olduktan sonra gerekli başvuruyu yaptığını dile getirdi.Reklamda 3 bin ile 10 bin lira arasında ihtiyaç kredisi verildiğini görünce mutlu olduğunu aktaran Gürel, şöyle konuştu",
        'Dünyaca ünlü piyanist Fazıl Say, 23 Nisanda Türkiye Büyük Millet Meclisinin (TBMM) kuruluşunun 100. yılı dolayısıyla yaptığı yeni besteyi sosyal medya hesaplarında paylaştı Say, Şükran Türküsü adını verdiği bestesini "23 Nisan Ulusal Egemenlik ve Çocuk Bayramı kutlu olsun" notuyla paylaştı.',
        'Menajer Ömer Uzun, Mohamed Elnenyin Beşiktaş ta kalıp kalmayacağıyla ilgili, "Gerek Arsenal tarafında gerek Beşiktaş tarafında salgın sonrasında birtakım şeyler beklemede. Şu ana kadar resmi bir görüşme yapamadık ancak şunu biliyorum ki gerek teknik heyet gerek Beşiktaş, Elnenyi bir sezon daha takımda tutmak istiyor. İşler biraz daha netleştikten sonra da Arsenal ile de oyuncunun gelecek sezon planlamasıyla ilgili birtakım görüşmeler yapılacaktır" dedi',
        "Yeni tip corona virüs salgını nedeniyle yaşanan ölümlerin sayısı 185 bini aşarken, bilim insanları ve ilaç firmaları küresel salgını durdurmak için birbiriyle yarışıyor. Dünya Sağlık Örgütü (DSÖ), dünya genelinde 83 yeni tip corona virüs (Covid-19), aşısının geliştirildiğini ve bunların altısının insan denemeleri aşamasında olduğunu belirtti."]

In [127]:
text1 = [unicode_tr(i).lower() for i in text1]

In [128]:
text1

["antalya'nın muratpaşa ilçesinde yaşayan emekli ahmet turan gürel de sunulan kredi avantajından haberdar olduktan sonra gerekli başvuruyu yaptığını dile getirdi.reklamda 3 bin ile 10 bin lira arasında ihtiyaç kredisi verildiğini görünce mutlu olduğunu aktaran gürel, şöyle konuştu",
 'dünyaca ünlü piyanist fazıl say, 23 nisanda türkiye büyük millet meclisinin (tbmm) kuruluşunun 100. yılı dolayısıyla yaptığı yeni besteyi sosyal medya hesaplarında paylaştı say, şükran türküsü adını verdiği bestesini "23 nisan ulusal egemenlik ve çocuk bayramı kutlu olsun" notuyla paylaştı.',
 'menajer ömer uzun, mohamed elnenyin beşiktaş ta kalıp kalmayacağıyla ilgili, "gerek arsenal tarafında gerek beşiktaş tarafında salgın sonrasında birtakım şeyler beklemede. şu ana kadar resmi bir görüşme yapamadık ancak şunu biliyorum ki gerek teknik heyet gerek beşiktaş, elnenyi bir sezon daha takımda tutmak istiyor. işler biraz daha netleştikten sonra da arsenal ile de oyuncunun gelecek sezon planlamasıyla ilgili 

In [129]:
text1 = [re.sub(r"[^a-zğüı'şöç]", " ", i) for i in text1]
text1

["antalya'nın muratpaşa ilçesinde yaşayan emekli ahmet turan gürel de sunulan kredi avantajından haberdar olduktan sonra gerekli başvuruyu yaptığını dile getirdi reklamda   bin ile    bin lira arasında ihtiyaç kredisi verildiğini görünce mutlu olduğunu aktaran gürel  şöyle konuştu",
 'dünyaca ünlü piyanist fazıl say     nisanda türkiye büyük millet meclisinin  tbmm  kuruluşunun      yılı dolayısıyla yaptığı yeni besteyi sosyal medya hesaplarında paylaştı say  şükran türküsü adını verdiği bestesini     nisan ulusal egemenlik ve çocuk bayramı kutlu olsun  notuyla paylaştı ',
 'menajer ömer uzun  mohamed elnenyin beşiktaş ta kalıp kalmayacağıyla ilgili   gerek arsenal tarafında gerek beşiktaş tarafında salgın sonrasında birtakım şeyler beklemede  şu ana kadar resmi bir görüşme yapamadık ancak şunu biliyorum ki gerek teknik heyet gerek beşiktaş  elnenyi bir sezon daha takımda tutmak istiyor  işler biraz daha netleştikten sonra da arsenal ile de oyuncunun gelecek sezon planlamasıyla ilgili 

In [130]:
text1=count_vectorizer.transform(text1)
text1=text1.toarray()

In [131]:
text1.shape

(4, 15000)

In [132]:
X_test.shape

(720, 15000)

In [133]:
predictions = nb.predict(text1)

In [134]:
mapping_list = list(mapping.keys())

In [135]:
for j in predictions:
    pred = mapping_list[j]
    print(pred)

ekonomi
kultursanat
spor
saglik
